# journalists-scraper

## Load Libraries

In [ ]:
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import requests
from bs4 import BeautifulSoup
from datetime import datetime
from datetime import timedelta
from datetime import date
import csv
import pandas as pd
import time
import json
import fnmatch
import os
import tabula
from tabula.io import read_pdf
import urllib.request
from bs4 import BeautifulSoup
import re
from geopy.geocoders import Nominatim
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

## Download csv

#### Download the [csv](https://cpj.org/data/killed/?status=Killed&motiveConfirmed%5B%5D=Confirmed&type%5B%5D=Journalist&start_year=1992&end_year=2021&group_by=year) from the CPJ website about journalists who were killed and load it in memory.

In [4]:
journalists_raw = pd.read_csv('journalists_killed.csv')
journalists_raw

,year,combinedStatus,fullName,sortName,primaryNationality,secondaryNationality,tertiaryNationality,gender,photoUrl,photoCredit,type,lastStatus,typeOfDeath,status,employedAs,organizations,jobs,coverage,mediums,country,location,region,state,locality,province,localOrForeign,sourcesOfFire,motiveConfirmed,accountabilityCrossfire,accountabilityAssignment,impunityMurder,tortured,captive,threatened,charges,motive,lengthOfSentence,healthProblems,impCountry,entry,sentenceDate,sentence,locationImprisoned
0,2008,Confirmed,Abdul Aziz Shaheen,NaN,Pakistan,NaN,NaN,Male,NaN,NaN,Journalist,NaN,Crossfire,Killed,Staff,Azadi,Print Reporter,"Politics,War",Print,Pakistan,Swat,NaN,NaN,Swat,NaN,Local,Military Officials,Confirmed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2017,Confirmed,Abdul Hakim Shimul,NaN,Bangladesh,NaN,NaN,Male,NaN,NaN,Journalist,Killed,Dangerous Assignment,Killed,Staff,Samakal,Print Reporter,Politics,Print,Bangladesh,Shahjadpur,NaN,NaN,Shahjadpur,NaN,Local,Government Officials,Confirmed,NaN,NaN,Complete Impunity,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2012,Confirmed,Abdul Haq Baloch,NaN,Pakistan,NaN,NaN,Male,NaN,NaN,Journalist,NaN,Murder,Killed,Staff,ARY Television,Broadcast Reporter,"Crime,Politics,War",Television,Pakistan,Khuzdar,NaN,NaN,Khuzdar,NaN,Local,Political Group,Confirmed,NaN,NaN,Complete Impunity,No,No,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2012,Confirmed,Abdul Qadir Hajizai,NaN,Pakistan,NaN,NaN,Male,NaN,NaN,Journalist,NaN,Murder,Killed,Freelance,WASH TV,Broadcast Reporter,"Politics,War",Television,Pakistan,Quetta,NaN,NaN,Quetta,NaN,Local,Political Group,Confirmed,NaN,NaN,Complete Impunity,No,No,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2008,Confirmed,Abdul Razzak Johra,NaN,Pakistan,NaN,NaN,Male,NaN,NaN,Journalist,NaN,Murder,Killed,Staff,Royal TV,Broadcast Reporter,Crime,Television,Pakistan,Punjab,NaN,NaN,Punjab,NaN,Local,Criminal Group,Confirmed,NaN,NaN,Complete Impunity,No,No,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2010,Confirmed,Abdul Wahab,NaN,Pakistan,NaN,NaN,Male,NaN,NaN,Journalist,NaN,Dangerous Assignment,Killed,Staff,Express News,"Broadcast Reporter,Camera Operator","Politics,War",Television,Pakistan,Ghalanai,NaN,NaN,Ghalanai,NaN,Local,Political Group,Confirmed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2018,Confirmed,Achyutananda Sahu,NaN,India,NaN,NaN,Male,https://cpj-photos.s3.us-west-2.amazonaws.com/...,Doordarshan,Journalist,Killed,Crossfire,Killed,Staff,Doordarshan,Camera Operator,Politics,Television,India,Chhattisgarh,NaN,Chhattisgarh,NaN,NaN,Local,Political Group,Confirmed,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2013,Confirmed,Ahmed Rajib Haider,NaN,Bangladesh,NaN,NaN,Male,NaN,NaN,Journalist,NaN,Murder,Killed,Freelance,Freelance,Internet Reporter,"Corruption,Human Rights,Politics",Internet,Bangladesh,Dhaka,NaN,NaN,Dhaka,NaN,Local,Political Group,Confirmed,NaN,NaN,Full Justice,No,No,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2014,Confirmed,Ahmed Rilwan Abdulla,NaN,Maldives,NaN,NaN,Male,https://cpj-photos.s3.us-west-2.amazonaws.com/...,CPJ,Journalist,Killed,Murder,Killed,Staff,Minivan News,"Internet Reporter,Print Reporter","Corruption,Crime,Culture,Human Rights,Politics","Print,Internet",Maldives,Male,NaN,NaN,Male,NaN,Local,Political Group,Confirmed,NaN,NaN,Complete Impunity,No,Yes,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2004,Confirmed,Aiyathurai Nadesan,NaN,Sri Lanka,NaN,NaN,Male,NaN,NaN,Journalist,NaN,Murder,Killed,Staff,Virakesari,Print Reporter,"Corruption,Human Rights,Politics",Print,Sri Lanka,Batticaloa,NaN,NaN,Batticaloa,NaN,Local,Military Officials,Confirmed,NaN,NaN,Complete Impunity,No,No,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Scraper

#### More information about each journalist can be retrieved from their bio page, which is in format 'https://cpj.org/data/people/< firstname-lastname >'

In [ ]:
# Create a list to store journalists' names
names_list = []
# Create a list of URLs that we want to scrape
url_list = []
# create a list of descriptions. We will use this to store the bios we retrieve.
desc_list = []
# Define base URL - the first part of the URL that's constant for every journalist's bio
base_url = "https://cpj.org/data/people/"
# Iterate through the dataframe to get each name
for index, row in journalists_raw.iterrows():
    names_list.append(row["fullName"])
# In order to get the desired URL, we have to add a dash (-) between first and last names
for name in names_list:
    name = name.replace(' ', '-')
    # Add this to our base URL defined above
    name_w_url = base_url + name
    # Append url_list with this newly constructed URL
    url_list.append(name_w_url)
# Iterate through each URL in url_list
for link in url_list:
    url = link
    # Open the URL
    html = urllib.request.urlopen(url)
    # Parse the html file
    htmlParse = BeautifulSoup(html, 'html.parser')
    # Iterate through all 'p' tags, clean them
    for para in htmlParse.find_all('article', {"class":"entry-content"}):
        para = para.text
        para = para.replace('Share this:TwitterFacebookWhatsAppLinkedInEmailTelegram ', '')
        para = para.strip()
        para = para.replace('.\n', '.<br>')
        para = para.split('<br>')
        # We want the first paragraph only, which tells us about the journalist; more cleaning
        para = para[0]
        para = para.replace('\n','')
        # Print for testing
        print((para))
        print('--')
    desc_text = para
    # Append these descriptions to the desc_list created above
    desc_list.append(desc_text)
journalists_with_desc = journalists_raw
# Create a column in our original df, and store these descriptions for each journalist
journalists_with_desc['desc'] = desc_list
# View the df
journalists_with_desc

#### Now, we have to add a new column to our df, which will have both city and country. This will be used to obtain coordinates for that location.

In [ ]:
# Journalists with city_country column

journalists_city_country = journalists_with_desc
journalists_city_country["city_country"] = journalists_city_country["location"] + ", "+ journalists_city_country["country"]

journalists_city_country

#### Some areas were listed as "an area outside..." or "an area near..." Take them out for consistency. 

In [ ]:
# clean that column using string detect
journalists_city_country_clean = journalists_city_country
journalists_city_country_clean['city_country'] = journalists_city_country_clean['city_country'].str.replace('an area outside ','')
journalists_city_country_clean['city_country'] = journalists_city_country_clean['city_country'].str.replace('an area near ','')
journalists_city_country_clean['city_country'] = journalists_city_country_clean['city_country'].str.replace('Malé','Male')


# journalists_city_country_clean.sort_values(by = ["city_country"], ascending=True)
journalists_city_country_clean

In [ ]:
# Store this in a dataframe called jourcoords
jourcoords = journalists_city_country_clean
jourcoords

#### Now, we use the geopy package to get coordinates. Some locations return errors, so we'll have to add coordinates manually. Add 'pass' if the function returns an error.

In [ ]:
# create an empty list list called place_list
place_list = []
for place in jourcoords["city_country"]:
    place_list.append(place)

# Create empty lists for latitude and longitude
lat_list = []
long_list = []
for item in place_list:
    # Try to get coordinates for locations
    try: 
        geolocator = Nominatim(user_agent="Aadit")
        location = geolocator.geocode(item)
        lat_list.append(location.latitude)
        long_list.append(location.longitude)
    # If the function errors out, return 'pass'
    except (RuntimeError, TypeError, NameError, AttributeError):
        lat_list.append('pass')
        long_list.append('pass')
# Print for testing
print(len(long_list))
print(len(lat_list))

In [ ]:
# Append lat and lon to jourcoords df
jourcoords["lat"] = lat_list
jourcoords['lon'] = long_list
jourcoords

## Export csv

#### Add coordinates for the four locations that returned 'pass'

In [ ]:
jourcoords.to_csv('final_file_clean.csv')